In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
from main import *

##### Fetch weights

In [4]:
weights = run(num_stocks=20, 
              num_days=60, 
              num_components=5)

Using $\epsilon$ = [0.0071295844770898195, 0.008177287694004049, 0.005710018249135276, 7.694215542022208e-05, 0.004547196039106853]$ for 5 components
Conic program object:
Number of variables:           3329
Continuous/binaries/integers:  3329/0/0
---------------------------------------------
Number of linear constraints:  25202
Inequalities/equalities:       25201/1
Number of coefficients:        98706
---------------------------------------------
Number of SOC constraints:     0
---------------------------------------------
Number of ExpCone constraints: 0
---------------------------------------------
Number of PSCone constraints:  0

Being solved by the default LP solver...
Solution status: 0
Running time: 0.2561s
Solver:          SciPy
Solution status: 0
Solution time:   0.2561s
-------------------------------------
Objective value: 0.14072660603895382
Primal solution: Available
Dual solution:   Available


In [5]:
weights

A       0.061893
AAL     0.035850
AAPL    0.061893
ABBV    0.061893
ABNB    0.061893
ABT     0.061893
ACGL    0.061893
ACN     0.061893
ADBE    0.000000
ADI     0.061893
ADM     0.061893
ADP     0.061893
ADSK    0.017262
AEE     0.061893
AEP     0.018489
AES     0.000000
AFL     0.061893
AIG     0.061893
AIZ     0.061893
AJG     0.061893
dtype: float64

##### Entropy ratio

In [13]:
m = 20
sum((weights * np.log(weights)).dropna()) / (m * (1 / m * np.log(1 / m)))

0.9501190654623692